In [1]:
!pip3 install ../../random-forest-mc/

Processing /work/random-forest-mc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for random-forest-mc: filename=random_forest_mc-1.0.3-py3-none-any.whl size=13683 sha256=ebae454773847f73f72a96da72377bafc8a330d52f9e4493de3dfc32b24fc53f
  Stored in directory: /root/.cache/pip/wheels/eb/5f/19/4722d7f6bf0d13102d1af6ab98e4b53771da1ed44ce678108e
Successfully built random-forest-mc

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [97]:
from random_forest_mc.model import RandomForestMC
from random_forest_mc.utils import LoadDicts
import pandas as pd
import numpy as np

from typing import Any
from typing import Dict
from typing import List
from typing import Optional
from typing import Tuple
from typing import Union

from numbers import Number
import logging as log

# Load dataset

In [84]:
dicts = LoadDicts("../../random-forest-mc/tests/")
dataset_dict = dicts.datasets_metadata
ds_name = "titanic"
params = dataset_dict[ds_name]
dataset = (
    pd.read_csv(params["csv_path"])[params["ds_cols"] + [params["target_col"]]]
    .dropna()
    .reset_index(drop=True)
)
dataset["Age"] = dataset["Age"].astype(np.uint8)
dataset["SibSp"] = dataset["SibSp"].astype(np.uint8)
dataset["Pclass"] = dataset["Pclass"].astype(str)
dataset["Fare"] = dataset["Fare"].astype(np.uint32)
ds_cols = params["ds_cols"]
target_col = params["target_col"]

# Load current model

In [24]:
cls = RandomForestMC(target_col=target_col)
cls.fit(dataset)
row = dataset.reset_index(drop=True).loc[0]
predict_row = cls.predict(row)
predict_ds = cls.predict(dataset.sample(n=10))
predict_probs_ds = cls.predict_proba(dataset.sample(n=10))

In [25]:
predict_ds

['0', '0', '1', '0', '0', '0', '0', '0', '0', '1']

In [26]:
cls

RandomForestMC(len(Forest)=16,n_trees=16,model_version=1.0.4-dev,module_version=1.0.4-dev)

# Generate some missing data

In [79]:
df_tmp = dataset.sample(frac=0.2).reset_index(drop=True)
mask_random = np.random.choice([True, False], size=df_tmp[ds_cols].shape, p=[0.7, 0.3])
dataset_missing_values = df_tmp[ds_cols].mask(~mask_random)
dataset_missing_values[target_col] = df_tmp[target_col]
dataset_missing_values_ground_truth = df_tmp

In [80]:
dataset_missing_values

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived
0,3,male,17.0,NaN,8.0,NaN,0
1,1,female,NaN,0.0,134.0,C,1
2,2,NaN,32.0,NaN,13.0,NaN,1
3,3,NaN,21.0,0.0,NaN,NaN,0
4,2,male,59.0,NaN,13.0,NaN,0
...,...,...,...,...,...,...,...
137,NaN,male,NaN,0.0,7.0,S,0
138,1,NaN,62.0,0.0,NaN,NaN,0
139,1,male,65.0,NaN,26.0,S,0
140,NaN,NaN,16.0,0.0,NaN,NaN,0


In [90]:
dict_values = {col: dataset[col].unique().tolist() for col in ds_cols}

In [91]:
dict_values.keys()

dict_keys(['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked'])

# Load the new extention

In [104]:
# a row of pd.DataFrame.iterrows()
# dsRow: TypeAlias = pd.core.series.Series
dsRow = pd.core.series.Series

# A tree composed by a assimetric tree of dictionaries:
# TypeTree: TypeAlias = Dict
TypeTree = Dict

# Value type of classes
# TypeClassVal: TypeAlias = Any
TypeClassVal = Any  # !Review if is not forced to be str!

# Type of the leaf
# TypeLeaf: TypeAlias = Dict[TypeClassVal, float]
TypeLeaf = Dict[TypeClassVal, float]

In [111]:
# How to format a dict with values to fill the missing ones
featName = str
featValue = Union[str, Number]
dictValues = Dict[featName, featValue]


class RandomForestMC_Ext(RandomForestMC):
    @staticmethod
    def _fill_row_missing(row: dsRow, dict_values : dictValues) -> pd.DataFrame:
        pass
        
        
    def predictMissingValues(self, row_or_matrix: Union[dsRow, pd.DataFrame], dict_values : dictValues, use_all_Tress: bool = True):
        used_features = set()
        for Tree in self:
            used_features |= set(Tree.used_features)
        not_have_feats = set(dict_values.keys()) - used_features
        if not_have_feats:
            _tmp = ", ".join(not_have_feats)
            log.warning(f'The Forest model have not the following feature(s): [{_tmp}].')
        
        if isinstance(row_or_matrix, dsRow):
            pass
        if isinstance(row_or_matrix, pd.DataFrame):
            pass

In [112]:
cls_ext = RandomForestMC_Ext(target_col=params["target_col"])
cls_ext.fit(dataset)

Planting the forest: 100%|██████████| 16/16 [00:06<00:00,  2.37it/s]


In [113]:
cls_ext

RandomForestMC_Ext(len(Forest)=16,n_trees=16,model_version=1.0.4-dev,module_version=1.0.4-dev)

In [114]:
cls_ext.__class__.__name__

'RandomForestMC_Ext'

In [115]:
cls_ext.predictMissingValues(dataset_missing_values, dict_values)